# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../flaviolin yield data/DBTL1/media_descriptions.csv` - media designs for each of the wells
   
   - `../flaviolin yield data/DBTL1/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 1

user_params = {
    'media_file': f'../flaviolin yield data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin yield data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin yield data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.003661,32.989243,0.144954,0.614290,0.017450,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05
B1,40.0,4.0,0.003661,32.989243,0.144954,0.614290,0.017450,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05
C1,40.0,4.0,0.003661,32.989243,0.144954,0.614290,0.017450,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05
D1,40.0,4.0,0.033185,10.948725,2.408075,2.381316,0.083375,6.352852,0.394845,42.424487,0.000010,0.001777,0.000277,0.000396,0.000053,0.05
E1,40.0,4.0,0.033185,10.948725,2.408075,2.381316,0.083375,6.352852,0.394845,42.424487,0.000010,0.001777,0.000277,0.000396,0.000053,0.05


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [5]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F3    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [6]:
column_order = True if df.index[1][0] == 'B' else False

In [7]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C1_WA1_C1-R1
C1_WA1_C1-R2
C1_WA1_C1-R3
C1_WD1_F1-R1
C1_WD1_F1-R2
C1_WD1_F1-R3
C1_WA2_C2-R1
C1_WA2_C2-R2
C1_WA2_C2-R3
C1_WD2_F2-R1
C1_WD2_F2-R2
C1_WD2_F2-R3
C1_WA3_C3-R1
C1_WA3_C3-R2
C1_WA3_C3-R3
C1_WD3_F3-R1
C1_WD3_F3-R2
C1_WD3_F3-R3
C1_WA4_C4-R1
C1_WA4_C4-R2
C1_WA4_C4-R3
C1_WD4_F4-R1
C1_WD4_F4-R2
C1_WD4_F4-R3
C1_WA5_C5-R1
C1_WA5_C5-R2
C1_WA5_C5-R3
C1_WD5_F5-R1
C1_WD5_F5-R2
C1_WD5_F5-R3
C1_WA6_C6-R1
C1_WA6_C6-R2
C1_WA6_C6-R3
C1_WD6_F6-R1
C1_WD6_F6-R2
C1_WD6_F6-R3
C1_WA7_C7-R1
C1_WA7_C7-R2
C1_WA7_C7-R3
C1_WD7_F7-R1
C1_WD7_F7-R2
C1_WD7_F7-R3
C1_WA8_C8-R1
C1_WA8_C8-R2
C1_WA8_C8-R3
C1_WD8_F8-R1
C1_WD8_F8-R2
C1_WD8_F8-R3


In [8]:
df

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.003661,32.989243,0.144954,0.614290,0.017450,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WA1_C1-R1
B1,40.0,4.0,0.003661,32.989243,0.144954,0.614290,0.017450,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WA1_C1-R2
C1,40.0,4.0,0.003661,32.989243,0.144954,0.614290,0.017450,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C1_WA1_C1-R3
D1,40.0,4.0,0.033185,10.948725,2.408075,2.381316,0.083375,6.352852,0.394845,42.424487,0.000010,0.001777,0.000277,0.000396,0.000053,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WD1_F1-R1
E1,40.0,4.0,0.033185,10.948725,2.408075,2.381316,0.083375,6.352852,0.394845,42.424487,0.000010,0.001777,0.000277,0.000396,0.000053,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WD1_F1-R2
F1,40.0,4.0,0.033185,10.948725,2.408075,2.381316,0.083375,6.352852,0.394845,42.424487,0.000010,0.001777,0.000277,0.000396,0.000053,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C1_WD1_F1-R3
A2,40.0,4.0,0.001096,129.695160,0.215986,5.430301,0.070543,9.628863,0.396082,21.949810,0.000185,0.000054,0.000027,0.000771,0.000028,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WA2_C2-R1
B2,40.0,4.0,0.001096,129.695160,0.215986,5.430301,0.070543,9.628863,0.396082,21.949810,0.000185,0.000054,0.000027,0.000771,0.000028,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WA2_C2-R2
C2,40.0,4.0,0.001096,129.695160,0.215986,5.430301,0.070543,9.628863,0.396082,21.949810,0.000185,0.000054,0.000027,0.000771,0.000028,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C1_WA2_C2-R3


## Process measurement files

Read measurements file:

In [9]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
A,1.0426,1.6266,0.4844,0.9276,0.2744,0.3686,1.7013,0.1831,0.0348,0.0444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,1.0403,1.6154,0.4914,0.9452,0.2664,0.3758,1.7681,0.2017,0.0416,0.0407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,1.0121,1.6062,0.4792,0.9410,0.2643,0.3600,1.6353,0.2326,0.0347,0.0362,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D,0.5032,1.1109,0.4801,0.9464,0.6865,0.2631,1.3853,0.8209,0.0291,0.0293,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E,0.5052,2.1220,0.4296,0.9248,0.6503,0.3096,1.4039,0.8188,0.0246,0.0242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F,0.4985,1.0021,0.3712,0.0222,0.6364,0.2816,1.3742,0.8072,0.0214,0.0212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G,0.0203,0.0204,0.0210,0.0209,0.0209,0.0207,0.0208,0.0209,0.0208,0.0203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
H,0.0217,0.0217,0.0214,0.0215,0.0216,0.0218,0.0210,0.0211,0.0215,0.0214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.2729,0.4975,0.3162,0.3454,0.2554,0.2299,0.5506,0.1407,0.1065,0.1016,0.0997,0.0970
B,0.2772,0.5072,0.3083,0.3362,0.2563,0.2411,0.5378,0.1431,0.1003,0.0977,0.0961,0.0927
C,0.2600,0.4714,0.3042,0.3376,0.2556,0.2254,0.6887,0.1383,0.0919,0.0933,0.0894,0.0888
D,0.2959,0.7116,0.1439,0.7301,0.3760,0.2034,0.8407,0.3794,0.0858,0.0856,0.0836,0.0792
E,0.3019,0.7473,0.1427,0.6687,0.3594,0.2033,0.8080,0.3693,0.0804,0.0794,0.0786,0.0735
F,0.3143,0.7009,0.1355,0.6523,0.3785,0.2024,0.7885,0.3621,0.0775,0.0759,0.0740,0.0704
G,0.0694,0.0730,0.0755,0.0769,0.0786,0.0784,0.0778,0.0780,0.0762,0.0753,0.0724,0.0692
H,0.0698,0.0710,0.0739,0.0745,0.0745,0.0740,0.0737,0.0740,0.0748,0.0734,0.0714,0.0694


### Process OD600

Normalize the data to the control well A9 value (water content):

In [11]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
A,1.0078,1.5918,0.4496,0.8928,0.2396,0.3338,1.6665,0.1483,0.0000,0.0096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,1.0055,1.5806,0.4566,0.9104,0.2316,0.3410,1.7333,0.1669,0.0068,0.0059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,0.9773,1.5714,0.4444,0.9062,0.2295,0.3252,1.6005,0.1978,-0.0001,0.0014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D,0.4684,1.0761,0.4453,0.9116,0.6517,0.2283,1.3505,0.7861,-0.0057,-0.0055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E,0.4704,2.0872,0.3948,0.8900,0.6155,0.2748,1.3691,0.7840,-0.0102,-0.0106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F,0.4637,0.9673,0.3364,-0.0126,0.6016,0.2468,1.3394,0.7724,-0.0134,-0.0136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G,-0.0145,-0.0144,-0.0138,-0.0139,-0.0139,-0.0141,-0.0140,-0.0139,-0.0140,-0.0145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
H,-0.0131,-0.0131,-0.0134,-0.0133,-0.0132,-0.0130,-0.0138,-0.0137,-0.0133,-0.0134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [12]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,10.078,15.918,4.496,8.928,2.396,3.338,16.665,1.483
B,10.055,15.806,4.566,9.104,2.316,3.410,17.333,1.669
C,9.773,15.714,4.444,9.062,2.295,3.252,16.005,1.978
D,4.684,10.761,4.453,9.116,6.517,2.283,13.505,7.861
E,4.704,20.872,3.948,8.900,6.155,2.748,13.691,7.840
F,4.637,9.673,3.364,0.000,6.016,2.468,13.394,7.724


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [13]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.1664,0.3910,0.2097,0.2389,0.1489,0.1234,0.4441,0.0342
B,0.1707,0.4007,0.2018,0.2297,0.1498,0.1346,0.4313,0.0366
C,0.1535,0.3649,0.1977,0.2311,0.1491,0.1189,0.5822,0.0318
D,0.1894,0.6051,0.0374,0.6236,0.2695,0.0969,0.7342,0.2729
E,0.1954,0.6408,0.0362,0.5622,0.2529,0.0968,0.7015,0.2628
F,0.2078,0.5944,0.0290,0.5458,0.2720,0.0959,0.6820,0.2556


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [14]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [15]:
measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.003661,32.989243,0.144954,0.61429,0.01745,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C1_WA1_C1-R1,0.1664
B1,40.0,4.0,0.003661,32.989243,0.144954,0.61429,0.01745,7.979403,0.270786,26.240883,0.000021,0.000706,0.000188,0.000546,0.000092,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C1_WA1_C1-R2,0.1707


## Create EDD Experiment Description File

In [16]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [17]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [18]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [1]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
df['Measurement Type'] = measurOD600

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD600]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

NameError: name 'user_params' is not defined

OD340

In [20]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [21]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

In [22]:
df['Kan[g/l]']

Well
A1    0.05
B1    0.05
C1    0.05
D1    0.05
E1    0.05
F1    0.05
A2    0.05
B2    0.05
C2    0.05
D2    0.05
E2    0.05
F2    0.05
A3    0.05
B3    0.05
C3    0.05
D3    0.05
E3    0.05
F3    0.05
A4    0.05
B4    0.05
C4    0.05
D4    0.05
E4    0.05
F4    0.05
A5    0.05
B5    0.05
C5    0.05
D5    0.05
E5    0.05
F5    0.05
A6    0.05
B6    0.05
C6    0.05
D6    0.05
E6    0.05
F6    0.05
A7    0.05
B7    0.05
C7    0.05
D7    0.05
E7    0.05
F7    0.05
A8    0.05
B8    0.05
C8    0.05
D8    0.05
E8    0.05
F8    0.05
Name: Kan[g/l], dtype: float64